
# Project Milestone One: Forming Your Team, Understanding the Problem, and Exploring the Data

#### **Due:** Midnight on October 26th (with 2-hour grace period) — **worth 25 points**

> **Note:** Because we must begin manual grading immediately, there will be *no* late period for this milestone.

This milestone is the first phase of your project. You’ll begin working in teams, select your dataset, perform basic exploratory data analysis (EDA), and frame your classification problem.  

1. **Form your project team.**  
   Convene your team and complete the **Team Contract** (available in your Homework Repository). This is **due on Sunday, October 19th** at midnight (along with Homework 07). Each member must review and sign it before submission.

2. **Select a team leader.**  
   Choose one team member to act as the **Gradescope submitter** for your team. The entire team should collaborate on the notebook, but only the leader will submit.

3. **Explore your dataset and frame your task.**  
   You’ll work through the notebook to  
   - Examine both provided datasets,  
   - Choose one for your project,
   - Be able to describe the classification problem you’ll be solving in business or applied terms,
   - Conduct basic EDA to understand its structure and challenges, and
   - Spot potential challenges, propose solutions, and select appropriate performance metrics.  

This milestone focuses on understanding your data and clearly articulating what your model will eventually predict. You are not required to build a model yet (that will happen in Milestone 2) but of course you have lots of example models to choose from previous Homeworks and Coding Notebooks and you may wish to explore a baseline model as you do this first phase of your project.


### The Datasets

The final project is a **classification task** using **one of two datasets**—one image-based and one text-based. These are the two
domains we have studied in detail, after learning the fundamentals in the first three weeks, and you have all you need to approach either of these datasets with confidence.

#### **1. Food-101 (Images)**  
A web-scraped collection of approximately **101,000 color photos** across **101 food categories** (≈ 800 train / 100 validation / 100 test per class).  
Images vary widely in **lighting, composition, and color balance**, making this dataset excellent for practicing **data cleaning**, **EDA**, and **augmentation** techniques such as random crops, flips, and color jitter.

#### **2. HuffPost News Category (Text)**  
Roughly **200,000 short news items** labeled into **41 topical categories** (e.g., *POLITICS*, *ENTERTAINMENT*, *PARENTING*).  
Each record contains a **headline**, a **short description**, which we will concatenate with a separator token to make a single text string:
> `"headline [SEP] short_description"`.
>
The `[SEP]` token simply marks where the headline ends and the description begins—mirroring conventions used in transformer models such as BERT.


### What To Do


We’ve provided template code to start your project:

* **Download** your selected dataset.
* **Visualize** a few representative samples (images or text excerpts).

After reviewing both datasets, you’ll **choose one** for your semester project.

In the sections that follow:

* **Problem One — Exploratory Data Analysis (EDA):**
  Quantify scale and structure, check class balance, and note any missing/duplicate or inconsistent entries.

* **Problem Two — Challenges & Solution Paths:**
  Identify likely issues (e.g., overlapping categories, imbalanced labels, data-quality problems, length/size variance) and outline practical remedies you would try. *(No model training required.)*

For tips on working with **Hugging Face Datasets** (helpful for large datasets), see the **Appendix**.

> **Important:** Keep only the section for the dataset you select and delete the other before submitting **Milestone 1**.


In [ ]:
# ============================================
# Useful Imports
# ============================================

# --- Standard Libraries
import os
import time
import math
import random
from collections import Counter

# --- Core Data / Numerics
import numpy as np
import pandas as pd

# --- Visualization
import matplotlib.pyplot as plt
# import seaborn as sns              # optional
import matplotlib.ticker as mticker  # optional (for formatted axes)

# --- NLP / Tokenization
import spacy                         # used for text preprocessing (HuffPost)

# --- Progress Tracking
from tqdm import tqdm                # optional (nice for loops)

from IPython.display import display

# --- TensorFlow / Keras (Deep Learning)
import tensorflow as tf
from tensorflow.keras import layers, models, Input, callbacks, regularizers, initializers
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.optimizers.schedules import CosineDecay, ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import (
    Dense, Dropout, Flatten, MaxPooling2D, Conv2D,
    SeparableConv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
)

# --- (Optional) Classical ML Baseline Tools
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score


# ============================================
# Global Configuration & Small Utilities
# ============================================

# Reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
tf.keras.utils.set_random_seed(random_seed)   # sets Python, NumPy, and TensorFlow seeds

# Utility: format seconds as HH:MM:SS

"""
Example usage to time your code:

start_time = time.time()
# ... your code here ...
print("Execution Time:", format_hms(time.time() - start_time))
"""

def format_hms(seconds: float) -> str:
    """Convert seconds to HH:MM:SS format."""
    return time.strftime("%H:%M:%S", time.gmtime(seconds))


In [ ]:
# If needed (in a new env):
#!pip install -U datasets pillow

In [ ]:
# --- Hugging Face Datasets
from datasets import load_dataset, DatasetDict
from datasets.features import ClassLabel

## Prelude: Choose your dataset and take a first look

You’ll pick **one** dataset—either **Food-101 (images)** or **HuffPost (text)**—and run the starter cells to confirm it loads, view class stats, and skim a few samples.

* **Food-101 (images):** 101 classes of web photos with **inconsistent lighting, white balance, color casts, and composition** (plus varying resolutions). These natural quirks make augmentation and input-pipeline choices meaningful.
* **HuffPost (text):** ~200k headline/summary pairs across 41 topics with **class imbalance** and some **category overlap**—good for trying stratified splits and reporting macro-F1 in addition to accuracy.

After you review the two sections below (download → minimal EDA → split), **keep one and delete the other.** Stick with your choice for the entire project.

> **Note:** We use Hugging Face **Dataset/DatasetDict** objects (table-like datasets), not NumPy/Pandas arrays. Most of what you need to know is contained in the template code, but see the **Appendix** for more information on HG Datasets.


---

### Dataset Two (Text): HuffPost Dataset

#### **Load HuffPost (headline + summary + category)**

**Note:** This loads a **Hugging Face `Dataset`**, not a NumPy array or Pandas DataFrame.
- Each record is a **dictionary** with fields such as `"headline"`, `"short_description"`, and `"category"`.
- You can access columns by name (e.g., `huff["headline"]`) and check dataset size with `len(huff)`.
- Treat it as a table of text fields — you’ll handle tokenization and vectorization later during preprocessing.
- The typical BERT-compatible separator is used to construct the sample texts

        `headline [SEP] short-description`



In [ ]:
# JSON mirror that preserves fields: headline, short_description, category, authors, link, date
URL = "https://huggingface.co/datasets/khalidalt/HuffPost/resolve/main/News_Category_Dataset_v2.json"
huff_all = load_dataset("json", data_files=URL, split="train")

print(huff_all)
print("Columns:", huff_all.column_names)
print("Total rows:", len(huff_all))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


News_Category_Dataset_v2.json:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['category', 'headline', 'authors', 'link', 'short_description', 'date'],
    num_rows: 200853
})
Columns: ['category', 'headline', 'authors', 'link', 'short_description', 'date']
Total rows: 200853


#### **Quick sanity checks (peek at a row)**

In [ ]:
ex = huff_all[0]
ex.keys()

dict_keys(['category', 'headline', 'authors', 'link', 'short_description', 'date'])

In [ ]:
ex = huff_all[0]
print("One example:")
print("  category          :", ex.get("category"))
print("  headline          :", (ex.get("headline") or "")[:120])
print("  authors           :", (ex.get("authors") or "")[:120])
print("  short_description :", (ex.get("short_description") or "")[:120])
print("  date              :", ex.get("date"))

One example:
  category          : CRIME
  headline          : There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV
  authors           : Melissa Jeltsen
  short_description : She left her husband. He killed their children. Just another day in America.
  date              : 2018-05-26 00:00:00


#### **Print 10 random samples (combined text with separator, no truncation)**

In [ ]:
# Show k random lines: "headline [SEP] short_description"

k = 10
seed = 7
rng = random.Random(seed)
idxs = rng.sample(range(len(huff_all)), k)

for i in idxs:
    ex = huff_all[i]
    print("  category          :", ex.get("category"))
    print("  headline          :", (ex.get("headline") or ""))
    print("  short_description :", (ex.get("short_description") or ""))
    print("  sample text       :", (ex.get("headline") or "")+'  [SEP]  '+ (ex.get("short_description") or ""))
    print()

  category          : ENTERTAINMENT
  headline          : Pregnant Kim Kardashian Rocks A Sheer Jumpsuit At The Airport
  short_description : Forget what you thought you knew about maternity style, because Kim's changing the game.
  sample text       : Pregnant Kim Kardashian Rocks A Sheer Jumpsuit At The Airport  [SEP]  Forget what you thought you knew about maternity style, because Kim's changing the game.

  category          : WOMEN
  headline          : How I Outran Misogyny
  short_description : "I have started running to retrain the way my brain sees my body, not as a sexual object but as a tool to get things done."
  sample text       : How I Outran Misogyny  [SEP]  "I have started running to retrain the way my brain sees my body, not as a sexual object but as a tool to get things done."

  category          : RELIGION
  headline          : A Prayer From the Mall of America
  short_description : I thank you for the Bloomington Police Department, and the Mall of America Security

#### **(Optional) Save splits to disk (reload later without re-splitting)**

We provide this in case you want to save the dataset to your local disk. Saving Food-101 splits to disk is not recommended unless you have ample local storage (it's huge!).

In [ ]:
# huff.save_to_disk("huffpost_splits")        # save
# from datasets import load_from_disk
# huff = load_from_disk("huffpost_splits")  # reload when needed

---

## Problem 1 – Choose the Dataset (10 pts)

#### Objective
In this problem, you will explore the two provided datasets and select one to use for your final project.  
Your goal is to understand the structure, content, and challenges of the dataset through basic exploratory data analysis (EDA).  
By the end of this Milestone notebook, you should be able to explain what makes the dataset interesting, identify potential modeling challenges (e.g., imbalance, ambiguity, quality issues), and justify why it is a good choice for your classification project.


#### What to Do
1. **Load both datasets** and examine the outputs of the template code provided.  
   After this brief inspection, **choose one dataset for your project** and **delete the template code for the other.**  For your chosen dataset, continue with the remaining steps.

2. **Inspect** the dataset's basic properties:  
   - **Number of samples and classes:**  
     Determine how many total examples and distinct categories are present. Verify that the counts match expectations (e.g., 101 food classes or 41 news topics).  
   - **Example records or images:**  
     View several samples to understand the input format, diversity, and potential quality issues.  
     For images, note lighting or composition differences; for text, read a few headlines and summaries to see how expressive they are (done for you in template code).
   - **Distribution of labels (check for imbalance):**  
     Plot or tabulate label frequencies to see whether some classes dominate. Imbalanced datasets can bias model training and may require special handling.  
   - **Missing or inconsistent data:**  
     Look for empty fields, unreadable images, duplicate entries, or mislabeled samples. Handle or document any issues you find.  
   - **Overlapping or ambiguous class labels:**  
     Identify categories that may not be clearly distinct—e.g., “apple pie” vs. “cheesecake,” or “POLITICS” vs. “WORLD NEWS.”  
     Ambiguity in labels can increase confusion between classes and reduce model accuracy.

3. **Visualize key aspects:**  
   Extend the template code to complete the EDA for your chosen dataset:  
   - **Images:**  
     Create visual summaries to better understand the data (some are implemented in the template code).  
     - Verify that the dataset is balanced across classes.  
     - Display a small grid of random images to check variation in appearance, composition, and background (done in template code).  
     - Compare image sizes to determine whether resizing or normalization will be needed.  
     - Examine lighting and color balance—many web-scraped photos vary widely in brightness, saturation, and white balance.  
   - **Text:**  
     Visualize class balance and linguistic properties (some implemented in the template code).  
     - Plot the number of samples per label to confirm class balance or imbalance.  
     - Compute basic text statistics such as average word count or vocabulary size.  
     - Examine examples for duplicates, near-duplicates, or entries that might fit multiple categories.  
     - *(Optional)* Generate a word-frequency plot or word cloud to highlight distinctive terms for a few classes.

4. **Answer the graded questions below.**


In [ ]:
# Your code here, add as many cells as you like. Provide organization and structure by appropriate comments, labels, and variable names. Explain what
# you are doing in case you come back to this in a year.



In [ ]:
from __future__ import annotations
import re, json, random, collections
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import matplotlib.pyplot as plt

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# helpers + deterministic seed
SEED = 7
random.seed(SEED); np.random.seed(SEED)

def tidy_plot(title="", xlabel="", ylabel=""):
    if title:  plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    plt.tight_layout()

@dataclass
class CFG:
    headline_col: str = "headline"
    desc_col: str     = "short_description"
    label_col: str    = "category"
    expect_classes: int = 41
    expect_min_samples: int = 100_000

cfg = CFG()

In [ ]:
# Build raw text per row

cols = set(getattr(huff_all, "column_names", []))
assert cfg.label_col in cols, f"Missing column: {cfg.label_col}"
have_headline = cfg.headline_col in cols
have_desc     = cfg.desc_col in cols

def _concat_text(example):
    h = example[cfg.headline_col] if have_headline else ""
    d = example[cfg.desc_col]     if have_desc     else ""
    example["text"] = f"{h} {d}".strip()
    return example

# Add a raw concatenated text column first
huff = huff_all.map(_concat_text)

Map:   0%|          | 0/200853 [00:00<?, ? examples/s]

In [ ]:
# 2.2 Get Samples

def _get_row(i):
    ex = huff_all[i]
    lab = ex.get(cfg.label_col, "")
    head = ex.get(cfg.headline_col, "") if have_headline else ""
    desc = ex.get(cfg.desc_col, "")     if have_desc else ""
    return lab, str(head), str(desc)

def _preview(s, n=140):
    s = s.replace("\n", " ").strip()
    return s if len(s) <= n else s[:n-1] + "…"

# 1) Random samples overall
print("\n=== Random Example Records (overall) ===")
idxs = random.sample(range(len(huff_all)), k=min(8, len(huff_all)))
for i in idxs:
    lab, head, desc = _get_row(i)
    flags = []
    if not head.strip(): flags.append("EMPTY_HEADLINE")
    if have_desc and not desc.strip(): flags.append("EMPTY_DESC")
    if "http" in head.lower() or "http" in desc.lower(): flags.append("URL_PRESENT")
    flag_str = f"  [{', '.join(flags)}]" if flags else ""
    print(f"[{lab}]{flag_str}")
    print("  headline:", _preview(head))
    if have_desc:
        print("  summary :", _preview(desc))
    print("-")

# 2) k examples per top m labels (by frequency)
from collections import Counter, defaultdict
labels_all = [huff_all[i][cfg.label_col] for i in range(len(huff_all))]
top_m = 3
k = 3
top_labels = [lab for lab, _ in Counter(labels_all).most_common(top_m)]

# Build index buckets
by_label = defaultdict(list)
for i, lab in enumerate(labels_all):
    by_label[lab].append(i)

print(f"\n=== {k} Examples per Top {top_m} Labels ===")
for lab in top_labels:
    print(f"\n-- {lab} --")
    picks = random.sample(by_label[lab], k=min(k, len(by_label[lab])))
    for i in picks:
        _, head, desc = _get_row(i)
        print("  headline:", _preview(head))
        if have_desc:
            print("  summary :", _preview(desc))
        print("  -----")


=== Random Example Records (overall) ===
[ENTERTAINMENT]
  headline: Pregnant Kim Kardashian Rocks A Sheer Jumpsuit At The Airport
  summary : Forget what you thought you knew about maternity style, because Kim's changing the game.
-
[WOMEN]
  headline: How I Outran Misogyny
  summary : "I have started running to retrain the way my brain sees my body, not as a sexual object but as a tool to get things done."
-
[RELIGION]
  headline: A Prayer From the Mall of America
  summary : I thank you for the Bloomington Police Department, and the Mall of America Security Force, for their willingness to keep things calm and pe…
-
[PARENTING]
  headline: What Happens After We Dry Our Tears?
  summary : It's in the weeks that follow tragedy that the majority of us look up from our hustle and bustle, look at strangers' faces and feel the bon…
-
[ENTERTAINMENT]
  headline: Luke Skywalker's Return To The Millennium Falcon Will Give You Chills
  summary : New "Star Wars" teaser features an emotional ho

In [ ]:
# 2.4 Missing / empty / duplicates on row data

num_samples = len(huff)
# Pull raw fields as lists (no pandas)
headlines = [example[cfg.headline_col] if have_headline else "" for example in huff]
descs     = [example[cfg.desc_col]     if have_desc     else "" for example in huff]
labels    = [example[cfg.label_col] for example in huff]

headline_null = sum(1 for h in headlines if h is None)
desc_null     = sum(1 for d in descs     if d is None)
label_null    = sum(1 for l in labels    if l is None)

headline_empty = sum(1 for h in headlines if (str(h or "").strip() == "")) if have_headline else None
desc_empty     = sum(1 for d in descs     if (str(d or "").strip() == "")) if have_desc     else None

# exact-duplicate key on (headline, short_description)
if have_headline and have_desc:
    keys = [(str(headlines[i] or "").strip().lower(), str(descs[i] or "").strip().lower()) for i in range(num_samples)]
else:
    # fallback to single available field
    keys = [(str(headlines[i] or "").strip().lower(),) for i in range(num_samples)] if have_headline else [(str(descs[i] or "").strip().lower(),) for i in range(num_samples)]

seen, dup_idx = set(), []
for i, k in enumerate(keys):
    if k in seen:
        dup_idx.append(i)
    else:
        seen.add(k)
dup_rate = (len(dup_idx) / num_samples) if num_samples else 0.0

print("\n=== Missing / Empty / Duplicates (raw fields) ===")
print("Missing:", json.dumps({
    "headline_null": headline_null if have_headline else None,
    "short_description_null": desc_null if have_desc else None,
    "category_null": label_null
}, indent=2))
print("Empties:", json.dumps({
    "headline_empty": headline_empty if have_headline else None,
    "short_description_empty": desc_empty if have_desc else None
}, indent=2))
print(f"Duplicate rows — approx rate: {dup_rate:.4f}")
if dup_idx:
    print("\nDuplicate examples (preview):")
    for j in dup_idx[:10]:
        lab = labels[j]
        print(f"[{lab}] {str(headlines[j])[:140]}")
        if have_desc:
            print(f"    {str(descs[j])[:140]}")
        print("-")


=== Missing / Empty / Duplicates (raw fields) ===
Missing: {
  "headline_null": 0,
  "short_description_null": 0,
  "category_null": 0
}
Empties: {
  "headline_empty": 6,
  "short_description_empty": 19712
}
Duplicate rows — approx rate: 0.0025

Duplicate examples (preview):
[PARENTS] The Funniest Tweets From Parents This Week
    Kids may say the darndest things, but parents tweet about them in the funniest ways. So each week, we round up the most hilarious
-
[ENTERTAINMENT] With This 'Shape Of Water' Clip, You'll Fall For Octavia Spencer In 60 Seconds
    Spencer and Sally Hawkins play 1960s janitors in Guillermo del Toro's wonderful new movie.
-
[POLITICS] Abortion Isn’t Main Reason More Republicans Still Won’t Back Doug Jones, New Poll Finds
    Would the Democrats’ Alabama Senate candidate do better if he were anti-abortion? Maybe not.
-
[PARENTS] 35 Hilarious Holiday Card Outtakes
    These kids were clearly not in the Christmas spirit.
-
[STYLE] The Worst Beauty Trends Of 2017


In [ ]:
# 2) spaCy Option B ( from Appendix)
#    - lemma, lower, alpha-only, remove stopwords
#    -> text_clean + len_tokens

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])
STOP = spacy.lang.en.stop_words.STOP_WORDS  # provided by milestone (note: not directly used below but available)

def spacy_clean(batch):
    docs = list(nlp.pipe(batch["text"], batch_size=1000))
    cleaned = []
    for doc in docs:
        tokens = [t.lemma_.lower() for t in doc if t.is_alpha and not t.is_stop]
        cleaned.append(" ".join(tokens))
    return {"text_clean": cleaned, "len_tokens": [len(c.split()) for c in cleaned]}

huff = huff.map(
    spacy_clean,
    batched=True,
    batch_size=1000,
    remove_columns=[c for c in huff.column_names if c not in ("category", "label", "text_clean", "len_tokens")]
)

# Pull cleaned fields for downstream EDA
labels      = list(huff[cfg.label_col])
texts_clean = list(huff["text_clean"])
len_tokens  = np.array(huff["len_tokens"], dtype=np.int32)

Map:   0%|          | 0/200853 [00:00<?, ? examples/s]

In [ ]:
# 2.1 & 2.2 Number of samples and classes + Distribution of labes (Basic properties & All label counts)

num_samples = len(labels)
classes = sorted(set(labels))
print("\n=== Basic Properties ===")
print(f"Total samples: {num_samples:,}")
print(f"Distinct classes: {len(classes)}")
print(f"Classes match expectation (~{cfg.expect_classes}): {len(classes) == cfg.expect_classes}")
print(f"Samples above minimum (~{cfg.expect_min_samples:,}): {num_samples >= cfg.expect_min_samples}")

# All labels - sorted by frequency
label_counts = collections.Counter(labels)
print("\n=== Label Distribution (ALL labels) ===")
for lab, cnt in label_counts.most_common():
    print(f"{lab:20s}  {cnt:,}")

# Plot All labels
items = label_counts.most_common()
labs_plot   = [lab for lab,_ in items][::-1]
counts_plot = [cnt for _,cnt in items][::-1]
plt.figure(figsize=(8, max(6, 0.25 * len(labs_plot))))
plt.barh(labs_plot, counts_plot)
tidy_plot(title="Label Distribution (All Labels)", xlabel="Count", ylabel="Label")
plt.show()

In [ ]:
# 4) Basix text stats (spaCy tokens) + histogram

if len_tokens.size:
    q = np.quantile(len_tokens, [0.5, 0.9, 0.95, 0.99])
    desc = {
        "count": int(len_tokens.size),
        "mean": float(len_tokens.mean()),
        "std": float(len_tokens.std(ddof=1)) if len_tokens.size > 1 else 0.0,
        "min": int(len_tokens.min()),
        "50%": float(q[0]),
        "90%": float(q[1]),
        "95%": float(q[2]),
        "99%": float(q[3]),
        "max": int(len_tokens.max()),
    }
else:
    desc = {}

print("\n=== Text Length Summary (spaCy tokens) ===")
print(json.dumps(desc, indent=2))

plt.figure()
plt.hist(len_tokens, bins=60)
tidy_plot(
    title="Token Count Distribution (spaCy lemma, alpha, no-stop)",
    xlabel="spaCy tokens per example",
    ylabel="Frequency"
)
plt.show()


In [ ]:
# 5) Vocabulary size (on text_clean)

vect = CountVectorizer(min_df=5)  # no stop_words here as spaCy already removed them
X_counts = vect.fit_transform(texts_clean)
vocab_size = len(vect.get_feature_names_out())
print("\n=== Vocabulary Size on text_clean (min_df=5) ===")
print(f"{vocab_size:,} unique terms")

In [ ]:
# 6) Overlapping labels (TF-IDF on text_clean)

print("\n=== Potentially Overlapping Label Pairs (TF-IDF on text_clean) ===")
by_class: Dict[str, List[str]] = collections.defaultdict(list)
for y, txt in zip(labels, texts_clean):
    by_class[y].append(txt)

class_labels = sorted(by_class.keys())
docs_per_class = [" ".join(by_class[y]) for y in class_labels]

if len(docs_per_class) >= 2:
    tfidf = TfidfVectorizer(min_df=5, max_features=50_000)
    Xc = tfidf.fit_transform(docs_per_class)
    sims = cosine_similarity(Xc)
    pairs = []
    for i in range(len(class_labels)):
        for j in range(i+1, len(class_labels)):
            pairs.append((class_labels[i], class_labels[j], float(sims[i, j])))
    pairs.sort(key=lambda x: x[2], reverse=True)
    for a, b, s in pairs[:12]:
        print(f"{a:20s}  vs  {b:20s}  cosine_sim={s:.4f}")
else:
    print("N/A (not enough classes)")

In [ ]:
# 7) Top TF-IDF terms per frequent class

tfidf_all = TfidfVectorizer(min_df=5, max_features=50_000)
tfidf_all.fit(texts_clean)
vocab = tfidf_all.get_feature_names_out()

def _top_terms_for_texts(texts: List[str], k: int = 15):
    X = tfidf_all.transform(texts)
    mean_vec = np.asarray(X.mean(axis=0)).ravel()
    idx = np.argsort(-mean_vec)[:k]
    terms  = [vocab[i] for i in idx]
    scores = [float(mean_vec[i]) for i in idx]
    return terms, scores

# Pick top 10 most frequent labels
top10 = [lab for lab,_ in label_counts.most_common(10)]
for lab in top10:
    docs = by_class[lab]
    if not docs:
        continue
    terms, scores = _top_terms_for_texts(docs, k=15)
    if not terms:
        continue
    plt.figure()
    plt.barh(terms[::-1], scores[::-1])
    tidy_plot(title=f"Top TF-IDF Terms — {lab} (on text_clean)", xlabel="Mean TF-IDF", ylabel="Term")
    plt.show()

In [ ]:
# Word Cloud for top 5 most occuring labels

try:
    from wordcloud import WordCloud
    # Pick top 5 most frequent labels
    top5_labels = [lab for lab,_ in label_counts.most_common(5)]
    for top_lab in top5_labels:
        if top_lab in by_class:
            # Use texts_clean data for the word cloud
            blob = " ".join(by_class[top_lab])
            wc = WordCloud(width=800, height=400, background_color="white").generate(blob)
            plt.figure(); plt.imshow(wc, interpolation="bilinear"); plt.axis("off")
            tidy_plot(title=f"Word Cloud — {top_lab} (on text_clean)")
            plt.show()
except Exception:
    print("\n(Word cloud skipped — install `wordcloud` to enable.)")

### Graded Questions (2 pts each)

For each question, answer thoroughly but concisely, in a short paragraph, longer or shorter as needed. Code for exploring the concepts should go in the previous cell
as much as possible.

1. **Dataset Summary:**  
   Describe your chosen dataset  (as if explaining to your *clueless boss* what you are working on).
   - State which dataset you are going to use.   
   - What kind of data does it contain (images or text)?  
   - How many samples and classes are there?  
   - What is the task you’ll perform (classification into what categories)?
   - What is the potential business use for this dataset?

1.1. **Your answer here:**

**- Dataset chosen**: HuffPost News Category

**- Type of data**: text

**- Size & classes**: 200,853 total samples; 41 distinct categories (e.g., POLITICS, WORLD NEWS, ENTERTAINMENT, BUSINESS, SPORTS, etc.).

**- Task**: multi-class text classification to predict the article’s category from its headline + short description.

**- Business use cases**:

- Content routing & curation: auto-tagging helps editors route articles to the right verticals and pages.

- Personalization: recommend sections/topics based on user interests.

- Analytics: track topic trends over time without manual labeling.

- Search: category aware retrieval/ranking.



2. **Initial Observations:**  
   What stood out to you from your EDA?  
   - Did you notice any imbalanced or ambiguous classes?  
   - Any patterns, anomalies, or potential sources of bias?  
   - For images: note any variation in lighting, composition, or color.  
   - For text: mention redundancy, topic overlap, or very short examples.

1.2. **Your answer here:**

**- Format & examples**: records are cleanly structured; headlines are punchy, descriptions add context. A quick sample scan shows a broad spread.

**- Class balance**: label counts are skewed (some big verticals dominate). This is typical for news: POLITICS / ENTERTAINMENT / WORLD NEWS are large; several niche categories are much smaller. I printed all label counts and saw a long tail.

**- Ambiguity/overlap**: PARENTING vs PARENTS, HEALTHY LIVING Vs WELLNESS

Food-related verticals can overlap. TF-IDF flagged such pairs as most similar.

**- Redundancy/short texts**: I found some empty fields (especially missing/blank descriptions in a minority of rows), a few duplicates (same headline+description), and many very short items (headlines with minimal descriptions). Token-length histograms (after spaCy cleaning) show a short-to-medium distribution with a long tail.




3. **Challenges & Implications:**  
   Based on your inspection, what challenges might affect model performance or training (e.g., imbalance, ambiguous labels, variable quality)?  

1.3. **Your answer here:**

**- Imbalanced Classes**: The label distribution shows significant imbalance. While "POLITICS" is the most frequent category with over 32,000 examples, several categories like "EDUCATION", "CULTURE & ARTS", and "LATINO VOICES" have around 1,000 or fewer examples. This imbalance will likely be a challenge for model training since models can overfit to majority classes and underperform on small one.

**- Short Examples**: The text length summary and histogram show that while the mean token count is around 15, the minimum is 0, and the 50th percentile is 15. There are a notable number of entries with very few tokens, including 19,714 empty short descriptions and 6 empty headlines. These short or empty examples might be less informative for classification.

**- Duplicates**: There is a small rate of duplicate rows based on the headline and short description (approximately 0.25%). While not a large percentage, these duplicates should be handled to prevent data leakage between splits.

**- Topic Overlap**: The TF-IDF centroid cosine similarity analysis indicates several pairs of categories with high similarity scores, suggesting potential overlap or ambiguity between topics like "PARENTING" and "PARENTS", "HEALTHY LIVING" and "WELLNESS", and "FOOD & DRINK" and "TASTE". This overlap might make it challenging for a model to distinguish clearly between these classes.





4. **Preparation Ideas:**  
   What data-cleaning or preprocessing steps might help address these issues?  
   (You will not implement these yet—just describe what you might do later.)

1.4. **Your answer here:**

**- Text preprocessing** (Appendix Option B): used spaCy to produce text_clean (lemma, alpha-only, remove stopwords) and len_tokens. we will keep using text_clean for TF-IDF and classical ML.

**- De-duplication & empties**:

- Drop exact duplicates by (headline, short_description).

- Remove rows with empty text_clean or with too-few tokens (min-length threshold).

**- Imbalance handling**:

- Stratified splits by label,

- Class weights or focal loss for linear/logistic baselines,

- Optionally downsample major classes for quick iterations.

**- Label hygiene**:

Keep a confusion-pair review loop or merge near-duplicates for a simpler baseline.

**- Features & models**:



5. **Reflection:**  
   Why did you choose this dataset over the other one?  
   - What makes it more interesting, realistic, or relevant for you?  
   - What do you expect to learn from working with it?

1.5. **Your answer here:**



## Problem 2 – Frame the Problem (15 pts)

#### Objective

Identify the **key challenges** in your chosen dataset and outline **practical solutions** you would try, plus how you’ll **evaluate** them later.

#### Steps to follow

1. **Diagnose likely challenges (from your EDA):**

   Examples:
   * **Class imbalance:**
     Report label counts and an imbalance ratio (max / median). List any minority classes.
   * **Length/size variance:**
     For text, show length percentiles (50/75/90/95) and estimate truncation rate at candidate `max_text_length`s (e.g., 256/300/512). For images, summarize native resolutions.
   * **Noise/duplicates/leakage:**
     Note empty or malformed items, near-duplicates, and how you would prevent cross-split leakage.
   * **Ambiguous/overlapping labels:**
     Give 2–3 example pairs you expect to be confusable and why.
   * **Compute constraints:**
     Briefly state limits (RAM/GPU/CPU) that might affect batch size, sequence length, or image size.

2. **Map each challenge to a concrete solution plan:**

   Examples:
   * **Imbalance →** `class_weight` or oversampling; report which one you’d try first and why.
   * **Length/size →** pick a target `max_text_length` (e.g., 95th percentile) with masking; for images, standardize resize/crop and basic augmentation.
   * **Noise/duplicates →** dedupe (hash/near-dup), drop empty/very short items, document any relabeling.
   * **Ambiguity →** consider merging labels (if justified), or add features (bigrams/char-ngrams; simple image augmentations).
   * **Overfitting risk →** early stopping on your primary metric, dropout/weight decay, freeze-then-finetune plan (for pretrained features).

3. **Explore appropriate evaluation metrics:**

   Examples:
   * **Primary metric:** pick one aligned to your data (e.g., **macro-F1** if imbalanced; accuracy if balanced).
   * **Secondary metric(s):** per-class precision/recall, confusion matrix.
   * **Protocol:** stratified Train/Val/Test (e.g., 70/15/15), fixed seed, leakage checks.

4. **Answer the graded questions below.**



In [ ]:
# Your code here, add as many cells as you like. Provide organization and structure by appropriate comments, labels, and variable names. Explain what
# you are doing in case you come back to this in a year.



### Graded Questions (3 pts each)

For each question, answer thoroughly but concisely, in a short paragraph, longer or shorter as needed. Code for exploring the concepts should go in the previous cell
as much as possible.

1. **State the prediction task**  
   - Describe what your model will predict (the *label*).  
   - *Examples:*  
     - “Given a photo of food, predict which of 101 categories it belongs to.”  
     - “Given a news headline + summary, predict its topical category.”  

2.1. **Your answer here:**

2. **Define inputs and outputs**  
   - *Inputs:* what information the model receives (e.g., pixel data, tokenized text).  
   - *Outputs:* the categorical label the model will predict.  

2.2. **Your answer here:**



3. **Identify possible challenges**  
   - Imbalanced classes, noisy data, ambiguous labels, overlapping features, or missing data  
   - *Images:* variation in lighting, color, composition, or size.  
   - *Text:* class imbalance, duplicate stories, short or ambiguous headlines.  

2.3. **Your answer here:**



4. **Propose how you will prepare or improve the data to address the challenges**  
   - *Images:* resizing, normalization, data augmentation (flips, rotations, brightness, color jitter).  
   - *Text:* tokenization, stop-word removal, TF-IDF, class balancing, embeddings (choose an embedding approach and specify its vector size).

2.4. **Your answer here:**



5. **Specify success metrics**  
   - Identify the metrics you plan to use to evaluate model performance—typically **accuracy** and/or **F1-score**, which are standard for classification tasks.  
   - Briefly explain **why** these metrics are appropriate for your dataset and goal. For instance, accuracy may suffice for well-balanced datasets, while F1-score better reflects performance when some classes are under-represented.
   - If your dataset is **imbalanced**, consider computing **per-class metrics** (e.g., precision, recall, or F1 for each label) or **macro-averaged** scores, which give equal weight to each class regardless of its size—ensuring that minority classes are evaluated fairly.
In some cases, weighted averages (which weight classes by their frequency) or **confusion matrices** can also provide useful insight.
> You haven't run any models yet, and we haven’t studied every possible metric, but you’re encouraged to ask your favorite generative AI tool which evaluation metrics might best fit your dataset!
   - Clearly state how you will interpret success—for example, “Our goal is to achieve at least 80% overall accuracy without large per-class disparities.”

2.5. **Your answer here:**



### Final Question: Describe what use you made of generative AI tools in preparing this Milestone.

**AI Question: Your answer here:**

---

## Appendix: A quick guide to Hugging Face Datasets

#### 1) What are they?

* A **table-like** dataset: rows = examples, columns = named fields (e.g., `"image"`, `"label"`, `"headline"`).
* Backed by **Apache Arrow** → fast, memory-efficient, lazy transforms.
* Two core objects:

  * `Dataset` — one table of rows/columns.
  * `DatasetDict` — a dict of splits, e.g. `{"train": Dataset, "val": Dataset, "test": Dataset}`.


#### 2) Load and inspect

```python
from datasets import load_dataset

# Food-101 (images)
food = load_dataset("food101", split="train+validation")  # both splits at once
len(food), food.column_names, food.features
# -> (≈101000, ['image','label'], {'label': ClassLabel(num_classes=101, names=[...])})

# Access by name (not by numeric column index!)
row0 = food[0]
img0, y0 = row0["image"], row0["label"]     # PIL image, int id
label_names = food.features["label"].names
label_names[y0]
```

For text (HuffPost JSON mirror):

```python
url = "https://huggingface.co/datasets/khalidalt/HuffPost/resolve/main/News_Category_Dataset_v2.json"
huff = load_dataset("json", data_files=url, split="train")
huff.column_names  # e.g. ['headline','short_description','category','authors','link','date']
```


#### 3) Common transforms

`Dataset`s are **immutable**: ops return a new dataset.

* **Map** (add/modify columns):

```python
def mk_text(ex):
    h = (ex.get("headline") or "").strip()
    s = (ex.get("short_description") or "").strip()
    return {"text": (h + " [SEP] " + s).strip()}

huff = huff.map(mk_text)  # adds 'text' column
```

* **Class-encode** labels (strings → integers with a vocabulary):

```python
from datasets.features import ClassLabel
if not isinstance(huff.features["category"], ClassLabel):
    huff = huff.class_encode_column("category")  # now ints with .names
```

* **Filter / select / rename / drop**:

```python
small = huff.select(range(5000))  # first 5k rows
huff = huff.remove_columns(["authors","link","date"])
huff = huff.rename_column("category", "label")
```


#### 4) Splitting & shuffling

```python
# Stratified 80/10/10 on Food-101 by 'label'
from datasets import DatasetDict
label_col = "label"

tmp = food.train_test_split(test_size=0.10, seed=42, stratify_by_column=label_col)
train_val = tmp["train"].train_test_split(test_size=1/9, seed=42, stratify_by_column=label_col)
ds = DatasetDict(train=train_val["train"], val=train_val["test"], test=tmp["test"])

len(ds["train"]), len(ds["val"]), len(ds["test"])
```

* `train_test_split` is **random by default** (reproducible with `seed=`).
* You typically **don’t need to pre-shuffle** datasets if your training dataloader already shuffles each epoch.


#### 5) Working with images

Use `with_transform` to apply on-the-fly resizing/augmentation and return tensors:

```python
import torchvision.transforms as T
from torch.utils.data import DataLoader
import torch, math, random

IM_SIZE = 224
train_tfms = T.Compose([T.RandomResizedCrop(IM_SIZE), T.RandomHorizontalFlip(), T.ToTensor()])
eval_tfms  = T.Compose([T.Resize(256), T.CenterCrop(IM_SIZE), T.ToTensor()])

def add_pixel_values(ex, tfms):  # ex['image'] -> ex['pixel_values']
    ex = dict(ex); ex["pixel_values"] = tfms(ex["image"]); return ex

train_t = ds["train"].with_transform(lambda ex: add_pixel_values(ex, train_tfms))
val_t   = ds["val"].with_transform(lambda ex: add_pixel_values(ex, eval_tfms))

def collate(batch):
    return {"pixel_values": torch.stack([b["pixel_values"] for b in batch]),
            "labels": torch.tensor([b["label"] for b in batch])}

train_loader = DataLoader(train_t, batch_size=64, shuffle=True,  collate_fn=collate)
val_loader   = DataLoader(val_t,   batch_size=64, shuffle=False, collate_fn=collate)
```

### 6) Working with Text

You can preprocess and tokenize text datasets using either a **transformer tokenizer** or a **linguistic pipeline like spaCy**, depending on your model type and goals.


#### Option A: Transformer Tokenizer (for fine-tuning models like BERT or DistilBERT)

Use a pretrained tokenizer with the Hugging Face `map` method to efficiently process your dataset in batches:

```python
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tok(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized = huff.map(
    tokenize,
    batched=True,
    remove_columns=[c for c in huff.column_names if c not in ("category", "label")]
)
```

This produces token IDs, attention masks, and other fields expected by transformer models.
Use this approach if your project involves fine-tuning pretrained language models.


#### Option B: spaCy Tokenization and Cleaning (for classical ML or custom preprocessing)

If you are **not using transformers**, or if you want to explore feature engineering (e.g., TF-IDF, word frequency, or embedding averages), you can preprocess text with **spaCy** instead.

```python
# !pip install spacy
# !python -m spacy download en_core_web_sm

import spacy
from datasets import load_dataset

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])
STOP = spacy.lang.en.stop_words.STOP_WORDS

def spacy_clean(batch):
    docs = list(nlp.pipe(batch["text"], batch_size=1000))
    cleaned = []
    for doc in docs:
        tokens = [t.lemma_.lower() for t in doc if t.is_alpha and not t.is_stop]
        cleaned.append(" ".join(tokens))
    return {"text_clean": cleaned, "len_tokens": [len(c.split()) for c in cleaned]}

huff = huff.map(
    spacy_clean,
    batched=True,
    batch_size=1000,
    remove_columns=[c for c in huff.column_names if c not in ("category", "label")]
)
```

The resulting column `text_clean` can be used with:

* `TfidfVectorizer` (Scikit-learn)
* `TextVectorization` (Keras)
* or any other custom embedding method.

> 💡 **Tip:** spaCy is ideal for lightweight NLP pipelines or for models that rely on explicit preprocessing (lemmatization, stopword removal). Transformer tokenizers, by contrast, expect *raw text* and handle subword tokenization internally.



#### 7) Slicing, concatenating, saving

```python
# Slicing
head_1000 = food.select(range(1000))
tail_10pct = food.select(range(int(0.9*len(food)), len(food)))

# Concatenate splits/datasets
from datasets import concatenate_datasets
all_train = concatenate_datasets([ds["train"], ds["val"]])

# Save / reload
ds.save_to_disk("food101_splits")
from datasets import load_from_disk
ds2 = load_from_disk("food101_splits")
```


#### 8) Quick “gotchas”

* **Columns by name** (strings), not numeric indices.
* Avoid converting huge columns to `list(...)` unless necessary; prefer vectorized ops with `map`, `filter`, `select`.
* `PYTHONHASHSEED` must be set **before** the Python process starts to matter; use explicit `seed=` arguments for reproducibility.
* Datasets print **previews** (e.g., `Column([6, 6, 6, ...])` is just the first few values).


#### 9) A minimal checklist to follow

1. `load_dataset(...)` → confirm `column_names`, `features`.
2. Build any needed columns (`"text"`), and **class-encode** labels if strings.
3. Make a **stratified 80/10/10** split (`train_test_split` ×2).
4. Do **EDA**: class counts, sample printouts or image grids.
5. For training:

   * **images** → `with_transform` + DataLoader
   * **text** → tokenizer via `.map(...)` + trainer/model pipeline
6. Save your `DatasetDict` with `save_to_disk(...)` (optional for text but handy, **don't** use it for big image datasets).

